In [88]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## 既存の帰属度、目的関数からFの値を算出

# 新たな評価関数をFとする  $$F=J+u_{ki}*d_{ii^*}$$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
#sysはコマンドライン引数
import sys
import csv

In [2]:
#クラスタ中心計算部分関数作成
def v_calculation(u_m,c,dimension,X):
    #分母計算(クラスタサイズになるはず)
    v_denominator = np.sum(u_m,axis=0)
    
    #分子計算
    v_numerator = np.zeros((c,dimension))
    for j in range(c):
        for i in range(X_size):
            v_numerator[j]+= u_m[i][j]*X[i]
   
    v = np.zeros((c,dimension ))
    for i in range(c):
        v[i] = v_numerator[i]/v_denominator[i]
    return v

## csvファイルから変数を抽出

In [3]:
#_______________入力ファイルやパラメータ指定部分_______________
c = 3

#正則化パラメーター設定
lam = 10
lam_1= 1/lam

m = 3
np.seterr(divide='ignore', invalid='ignore')
#csvファイルの読み込み
df = pd.read_csv("Iris-m3.csv")
#__________________________________________
print(df.head())
df = df[:150]
J = df.J[0]
F = df.F[0]
print("J",J)
print("F",F)

#Uの抽出
u_pd = []
for i in range(c):
    u_pd.append("u{}".format(i))
df_u = df.loc[:,u_pd]
u = df_u.values

#Xの抽出
df_X = df.loc[:,"sepal_length":"petal_width"]
# df_X = df.loc[:,"0":"1"]

#dfをnumpyで扱えるnp.ndarray変換
X = df_X.values
X = X.astype('float64')
#配列の行数、列数の格納
X_size,dimension = X.shape
cluster = df["cluster"].values

u_m = u**m
v = v_calculation(u_m,c,dimension,X)

   sepal_length  sepal_width  petal_length  petal_width  cluster        u0  \
0           5.1          3.5           1.4          0.2      1.0  0.060611   
1           4.9          3.0           1.4          0.2      1.0  0.108113   
2           4.7          3.2           1.3          0.2      1.0  0.101756   
3           4.6          3.1           1.5          0.2      1.0  0.122551   
4           5.0          3.6           1.4          0.2      1.0  0.070286   

         u1        u2          J           F  
0  0.885228  0.054161  29.289736  835.958407  
1  0.810330  0.081557  29.289736  835.958407  
2  0.820053  0.078190  29.289736  835.958407  
3  0.786852  0.090597  29.289736  835.958407  
4  0.870102  0.059612  29.289736  835.958407  
J 29.289735663504533
F 835.9584066789748


## Fの算出セル

In [88]:
u_m = u**m
v = v_calculation(u_m,c,dimension,X)

#中心間距離の作成
center_distance = np.zeros((c,c))
for i in range(c):
        center_distance[i]= np.sum((v[i]-v)**2,axis=1)
print("center_distance")
print(center_distance)
#dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
d = np.zeros((X_size,c))
for i in range(X_size):
    d[i] = np.sum((X[i]-v)**2,axis=1)
print("d_{ki}")
print(d[0])

#ここが怪しい
for i in range(c):
    d[cluster == i] = d[cluster == i]+center_distance[i]
print("d_{ki}+d_{ii*}")
print(d[0])

#評価関数計算
F = 0
for i in range(X_size):
    F += np.sum(u[i] * d[i])
# for i in range(X_size):
#     for j in range(c):
#         F += u[i][j] * d[i][j]
        
        
#この初期値のときの結果が優れている場合，bestを更新
print("Jの値:",J)
print("Fの値:",F)

center_distance
[[ 0.         10.49720232  2.45073051]
 [10.49720232  0.         22.12095316]
 [ 2.45073051 22.12095316  0.        ]]
d_{ki}
[11.11955116  0.03304539 22.8112015 ]
d_{ki}+d_{ii*}
[2.16167535e+01 3.30453901e-02 4.49321547e+01]
Jの値: 29.289735663504533
Fの値: 835.9584066789752


##　関数化

In [94]:
#引数(クラスタ数(c),クラスター(cluster),データセット(X),クラスター中心(v)、\
#ファジィ化パラム(m)、帰属度(u))

def calc_F(c,cluster,X,v,m,u):
    #中心間距離の作成
    center_distance = np.zeros((c,c))
    for i in range(c):
        center_distance[i]= np.sum((v[i]-v)**2,axis=1)
    print("center_distance")
    print(center_distance)
    #dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
    d = np.zeros((X_size,c))
    for i in range(X_size):
        d[i] = np.sum((X[i]-v)**2,axis=1)
    # print("d_{ki}")
    # print(d[0])

    new_d = np.zeros((X_size,c))
    #ここが怪しい
    for i in range(c):
        new_d[cluster == i] = d[cluster == i]+center_distance[i]
    # print("new_d{ki} = d_{ki}+d_{ii*}")
    # print(new_d[0])

    #評価関数計算(ついでにJも再計算してあってるか確認)
    F = 0
    kakunin_J = 0
    u_m = u**m
    for i in range(X_size):
        F += np.sum(u[i] * new_d[i])
        kakunin_J += np.sum(u_m[i] * d[i])


    #この初期値のときの結果が優れている場合，bestを更新
    print("Jの値:",J)
    print("Jの値を再計算:",kakunin_J)
    print("Fの値:",F)
    return F

In [95]:
F = calc_F(c,cluster,X,v,m,u)

center_distance
[[ 0.         10.49720232  2.45073051]
 [10.49720232  0.         22.12095316]
 [ 2.45073051 22.12095316  0.        ]]
Jの値: 29.289735663504533
Jの値を再計算: 29.289735663503222
Fの値: 835.9584066789752


## csvファイルへの書き出し

In [101]:
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F
0,5.1,3.5,1.4,0.2,1.0,0.060611,0.885228,0.054161,29.289736,835.958407
1,4.9,3.0,1.4,0.2,1.0,0.108113,0.810330,0.081557,29.289736,835.958407
2,4.7,3.2,1.3,0.2,1.0,0.101756,0.820053,0.078190,29.289736,835.958407
3,4.6,3.1,1.5,0.2,1.0,0.122551,0.786852,0.090597,29.289736,835.958407
4,5.0,3.6,1.4,0.2,1.0,0.070286,0.870102,0.059612,29.289736,835.958407
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0,0.181321,0.054109,0.764570,29.289736,835.958407
146,6.3,2.5,5.0,1.9,2.0,0.396489,0.088777,0.514734,29.289736,835.958407
147,6.5,3.0,5.2,2.0,2.0,0.173925,0.041181,0.784894,29.289736,835.958407
148,6.2,3.4,5.4,2.3,2.0,0.257683,0.084530,0.657787,29.289736,835.958407


In [610]:
df = pd.read_csv("fecbo-E-10-iris.csv")
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2
0,5.1,3.500000,1.4,0.2,1.0,0.060611,0.885228,0.054161
1,4.9,3.000000,1.4,0.2,1.0,0.108113,0.810330,0.081557
2,4.7,3.200000,1.3,0.2,1.0,0.101756,0.820053,0.078190
3,4.6,3.100000,1.5,0.2,1.0,0.122551,0.786852,0.090597
4,5.0,3.600000,1.4,0.2,1.0,0.070286,0.870102,0.059612
...,...,...,...,...,...,...,...,...
147,6.5,3.000000,5.2,2.0,2.0,0.173925,0.041181,0.784894
148,6.2,3.400000,5.4,2.3,2.0,0.257683,0.084530,0.657787
149,5.9,3.000000,5.1,1.8,2.0,0.432544,0.091590,0.475866
150,J,29.289736,NaN,NaN,NaN,NaN,NaN,NaN


## iris-m3

In [466]:
#_______________入力ファイルやパラメータ指定部分_______________
c = 3
m = 3
np.seterr(divide='ignore', invalid='ignore')
#csvファイルの読み込み
df = pd.read_csv("fecbo-db-m3-iris.csv")
#__________________________________________
df = df.iloc[:151]

J = df.iloc[-1][1]
df = df.iloc[:-1]

u_pd = []
for i in range(c):
    u_pd.append("u{}".format(i))
df_u = df.loc[:,u_pd]
u = df_u.values

df_X = df.loc[:,"sepal_length":"petal_width"]
# df_X = df.loc[:,"0":"1"]

#dfをnumpyで扱えるnp.ndarray変換
X = df_X.values
X = X.astype('float64')
#配列の行数、列数の格納
X_size,dimension = X.shape
cluster = df["cluster"].values

u_m = u**m
v = v_calculation(u_m,c,dimension,X)

#中心間距離の作成
center_distance = np.zeros((c,c))
for i in range(c):
    for j in range(c):
        center_distance[i][j]= np.sum((v[i]-v[j])**2)
print("center_distance")
print(center_distance)
#dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
d=[]
for i in range(X_size):
    for j in range(c):
        d.append(np.sum((X[i]-v[j])**2))
d = np.array(d)
d = d.reshape([X_size,c])
print("d_{ki}")
print(d[0])
for i in range(c):
    d[cluster == i] = d[cluster == i]+center_distance[i]
print("d_{ki}+d_{ii*}")
print(d[0]) 

#評価関数計算
F = 0
for i in range(X_size):
    for j in range(c):
        F += u[i][j] * d[i][j]
        
        
#この初期値のときの結果が優れている場合，bestを更新
print("Jの値:",J)
print("Fの値:",F)

center_distance
[[ 0.         10.49720232  2.45073051]
 [10.49720232  0.         22.12095316]
 [ 2.45073051 22.12095316  0.        ]]
d_{ki}
[11.11955116  0.03304539 22.8112015 ]
d_{ki}+d_{ii*}
[2.16167535e+01 3.30453901e-02 4.49321547e+01]
Jの値: 29.289735663504533
Fの値: 835.9584066789748


In [467]:
m3 = {}
m3["J"] = J
m3["F"] = F
m3

{'J': 29.289735663504533, 'F': 835.9584066789748}

In [469]:
print(np.sum(u,axis=0))
print(np.sum(u,axis=1))

[50. 50. 50.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


## iris-m5

In [470]:
#_______________入力ファイルやパラメータ指定部分_______________
c = 3
m = 5
np.seterr(divide='ignore', invalid='ignore')
#csvファイルの読み込み
df = pd.read_csv("fecbo-db-m5-iris.csv")
#__________________________________________
df = df.iloc[:151]

J = df.iloc[-1][1]
df = df.iloc[:-1]

u_pd = []
for i in range(c):
    u_pd.append("u{}".format(i))
df_u = df.loc[:,u_pd]
u = df_u.values

df_X = df.loc[:,"sepal_length":"petal_width"]
# df_X = df.loc[:,"0":"1"]

#dfをnumpyで扱えるnp.ndarray変換
X = df_X.values
X = X.astype('float64')
#配列の行数、列数の格納
X_size,dimension = X.shape
cluster = df["cluster"].values

u_m = u**m
v = v_calculation(u_m,c,dimension,X)

#中心間距離の作成
center_distance = np.zeros((c,c))
for i in range(c):
    for j in range(c):
        center_distance[i][j]= np.sum((v[i]-v[j])**2)
print("center_distance")
print(center_distance)
#dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
d=[]
for i in range(X_size):
    for j in range(c):
        d.append(np.sum((X[i]-v[j])**2))
d = np.array(d)
d = d.reshape([X_size,c])
print("d_{ki}")
print(d[0])
for i in range(c):
    d[cluster == i] = d[cluster == i]+center_distance[i]
print("d_{ki}+d_{ii*}")
print(d[0])

#評価関数計算
F = 0
for i in range(X_size):
    for j in range(c):
        F += u[i][j] * d[i][j]
        
        
#この初期値のときの結果が優れている場合，bestを更新
print("Jの値:",J)
print("Fの値:",F)

center_distance
[[ 0.         10.89473071  1.79795312]
 [10.89473071  0.         20.84956998]
 [ 1.79795312 20.84956998  0.        ]]
d_{ki}
[11.58065984  0.03437582 21.63188804]
d_{ki}+d_{ii*}
[2.24753906e+01 3.43758222e-02 4.24814580e+01]
Jの値: 4.254850237086912
Fの値: 1454.160086057192


In [471]:
m5 = {}
m5["J"] = J
m5["F"] = F
m5

{'J': 4.254850237086912, 'F': 1454.160086057192}

In [472]:
print(np.sum(u,axis=0))
print(np.sum(u,axis=1))

[50. 50. 50.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


## iris-m8

In [473]:
#_______________入力ファイルやパラメータ指定部分_______________
c = 3
m = 8
np.seterr(divide='ignore', invalid='ignore')
#csvファイルの読み込み
df = pd.read_csv("fecbo-db-m8-iris.csv")
#__________________________________________
df = df.iloc[:151]

J = df.iloc[-1][1]
df = df.iloc[:-1]

u_pd = []
for i in range(c):
    u_pd.append("u{}".format(i))
df_u = df.loc[:,u_pd]
u = df_u.values

df_X = df.loc[:,"sepal_length":"petal_width"]
# df_X = df.loc[:,"0":"1"]

#dfをnumpyで扱えるnp.ndarray変換
X = df_X.values
X = X.astype('float64')
#配列の行数、列数の格納
X_size,dimension = X.shape
cluster = df["cluster"].values

u_m = u**m
v = v_calculation(u_m,c,dimension,X)

#中心間距離の作成
center_distance = np.zeros((c,c))
for i in range(c):
    for j in range(c):
        center_distance[i][j]= np.sum((v[i]-v[j])**2)
print("center_distance")
print(center_distance)
#dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
d=[]
for i in range(X_size):
    for j in range(c):
        d.append(np.sum((X[i]-v[j])**2))
d = np.array(d)
d = d.reshape([X_size,c])
print("d_{ki}")
print(d[0])
for i in range(c):
    d[cluster == i] = d[cluster == i]+center_distance[i]
print("d_{ki}+d_{ii*}")
print(d[0])

#評価関数計算
F = 0
for i in range(X_size):
    for j in range(c):
        F += u[i][j] * d[i][j]
        
        
#この初期値のときの結果が優れている場合，bestを更新
print("Jの値:",J)
print("Fの値:",F)

center_distance
[[ 0.         19.90953107  1.37086191]
 [19.90953107  0.         11.23420833]
 [ 1.37086191 11.23420833  0.        ]]
d_{ki}
[20.71594016  0.03219863 11.93817815]
d_{ki}+d_{ii*}
[4.06254712e+01 3.21986279e-02 2.31723865e+01]
Jの値: 0.18012164024442
Fの値: 1785.6306597921155


In [474]:
m8 = {}
m8["J"] = J
m8["F"] = F
m8

{'J': 0.18012164024442, 'F': 1785.6306597921155}

In [475]:
print(np.sum(u,axis=0))
print(np.sum(u,axis=1))

[50. 50. 50.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


## まとめ

In [476]:
print(m3)
print(m5)
print(m8)

{'J': 29.289735663504533, 'F': 835.9584066789748}
{'J': 4.254850237086912, 'F': 1454.160086057192}
{'J': 0.18012164024442, 'F': 1785.6306597921155}
